In [1]:
import itertools
import json
 
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

In [4]:

with open("data/user_data.json", "r") as file_handler:    
    userdata = json.load(file_handler)


In [8]:
for post in userdata['posts']:
    msg = post['message'].split()
    print(findsubsets(msg, 2))

[('Text', 'content'), ('Text', 'for'), ('Text', 'the'), ('Text', 'post.'), ('Text', 'All'), ('Text', 'images'), ('Text', 'here'), ('Text', 'are'), ('Text', 'randomly'), ('Text', 'generated'), ('Text', 'by'), ('Text', 'picsum.photos'), ('Text', 'or'), ('Text', 'pravatar.cc'), ('content', 'for'), ('content', 'the'), ('content', 'post.'), ('content', 'All'), ('content', 'images'), ('content', 'here'), ('content', 'are'), ('content', 'randomly'), ('content', 'generated'), ('content', 'by'), ('content', 'picsum.photos'), ('content', 'or'), ('content', 'pravatar.cc'), ('for', 'the'), ('for', 'post.'), ('for', 'All'), ('for', 'images'), ('for', 'here'), ('for', 'are'), ('for', 'randomly'), ('for', 'generated'), ('for', 'by'), ('for', 'picsum.photos'), ('for', 'or'), ('for', 'pravatar.cc'), ('the', 'post.'), ('the', 'All'), ('the', 'images'), ('the', 'here'), ('the', 'are'), ('the', 'randomly'), ('the', 'generated'), ('the', 'by'), ('the', 'picsum.photos'), ('the', 'or'), ('the', 'pravatar.cc'

In [10]:
data = "You and I are stupid!"

In [11]:
import numpy as np
def word_combinations(sentence):
    data = sentence.split()
    out = []
    for j in range(0, len(data)+1):
        for i in range(0, j):
            data0 = data.copy()
            words = data0[i:j]

            for w in words: data0.remove(w)
            out.append(w)
            out.append(" ".join(data0))
    
    #for i in range(len(out)):
    #    if len(out[i])> 1:
    #        out[i] = " ".join(out[i])
    out =  list(np.unique(out))
    out.remove('')
    return out


from detoxify import Detoxify

model = Detoxify('original', device='cpu')
words = word_combinations(data)
print(words)



['I', 'I are stupid!', 'You', 'You I are stupid!', 'You and', 'You and I', 'You and I are', 'You and I stupid!', 'You and are stupid!', 'You and stupid!', 'You are stupid!', 'You stupid!', 'and', 'and I are stupid!', 'are', 'are stupid!', 'stupid!']


In [12]:
preds = []

for word in words:
    pred = model.predict(word)
    preds.append(pred)
        

In [13]:
pred_0 = {'toxicity':0, 'severe_toxicity':0, 'obscene':0, 'threat': 0, 'insult':0, 'identity_hate':0}
scores = {}
for word in data.split():
    scores[word] = pred_0.copy()

for word, pred in zip(words, preds):
    #print(word)
    for w in word.split():
        for cat in pred_0.keys():
            scores[w][cat] += pred[cat]/len(words)
            #print(w, cat, pred[cat])
    


In [14]:

import pandas as pd
df = pd.DataFrame(scores)


print(data, model.predict(data))
print(df)
print(df.sum(axis=0))

You and I are stupid! {'toxicity': 0.98052955, 'severe_toxicity': 0.016882569, 'obscene': 0.66394055, 'threat': 0.00091078173, 'insult': 0.8349678, 'identity_hate': 0.0047303983}
                      You       and         I       are   stupid!
toxicity         0.372228  0.229912  0.223315  0.338818  0.570357
severe_toxicity  0.012488  0.004580  0.002734  0.007849  0.017387
obscene          0.256587  0.148294  0.133173  0.220379  0.395823
threat           0.000573  0.000392  0.000375  0.000443  0.000767
insult           0.303814  0.145888  0.095971  0.235431  0.428720
identity_hate    0.003464  0.001440  0.000699  0.002506  0.004516
You        0.949154
and        0.530506
I          0.456268
are        0.805428
stupid!    1.417570
dtype: float64


In [15]:
model.predict('i hate you')

{'toxicity': 0.95099854,
 'severe_toxicity': 0.007945027,
 'obscene': 0.023724578,
 'threat': 0.039169792,
 'insult': 0.137607,
 'identity_hate': 0.02070892}

In [19]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)
twenty_test = fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

vec = TfidfVectorizer(min_df=3, stop_words='english',
                      ngram_range=(1, 2))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(lsa, clf)
pipe.fit(twenty_train.data, twenty_train.target)
pipe.score(twenty_test.data, twenty_test.target)


0.8901464713715047

In [21]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(twenty_train.target_names, y_pred):
        print("{:.3f} {}".format(prob, target))

doc = twenty_test.data[0]
print_prediction(doc)

0.001 alt.atheism
0.001 comp.graphics
0.996 sci.med
0.003 soc.religion.christian


In [23]:
import eli5
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=twenty_train.target_names)

/Users/mansur/Work/Projects/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
